In [1]:
# -*- coding: utf-8 -*-

"""
note something here

"""

__author__ = 'Wang Chen'
__time__ = '2019/7/15'

import copy
from queue import PriorityQueue

import pandas as pd
import numpy as np

from Item import Item
from Event import Event
from Hypercube import Hypercube
#from static import *


In [2]:
class Popcache:
    """
    dim: 维度
    item_list: 视频列表，主键为视频的id
    hypercube_array: 立方空间的dim维数组
    max_day: 最大天数
    cur_day: 当天天数
    all_UITs: np.adarray 从pandas转化来的原始值
    event_lastday_list: 前一天的事件缓存列表
    event_count: 事件计数
    cache_size: 缓存列表大小
    cache_list: 缓存列表，存储视频id
    cache_set:

    if_disp: 是否展示
    """
    def __init__(self, dim, max_item_id, max_day, cur_day, all_UITs, cache_size, if_disp):
        """
        dim: 维度
        max_item_id: 最大视频id
        max_day: 最大天数
        cur_day: 当天天数
        all_UITs: np.adarray 从pandas转化来的原始值
        event_lastday_list: 前一天的事件缓存列表
        cache_size: 缓存列表大小

        if_disp: 是否展示
        """
        self.dim = dim
        self.item_list = [Item(item_id) for item_id in range(max_item_id)]
        self.hypercube_array = self.init_hypercube_array(dim)
        self.max_day = max_day
        self.cur_day = cur_day
        self.all_UITs = all_UITs
        self.curday_values = None
        self.event_lastday_list = []
        self.event_count = -1
        self.cache_size = cache_size
        self.cache_list = PriorityQueue()
        self.cache_set = set()

        self.if_disp = if_disp

    def init_hypercube_array(self, dim):
        """
        构建hypercube空间
        :param dim:
        :return:
        """
        hypercube_array = [Hypercube() for _ in range(2)]
        for _ in range(dim - 1):
            hypercube_array = [copy.deepcopy(hypercube_array) for _ in range(2)]
        return hypercube_array

    def enumerate_hypercubes(self):
        hypercubes_list = self.hypercube_array
        for depth in range(self.dim-1):
            tmp_list = []
            for element in hypercubes_list:
                tmp_list.extend(element)
            hypercubes_list = tmp_list

        if self.if_disp:
            print("enumerate {} depth, total length {}".format(dim, hypercubes_list.__len__()))

        return hypercubes_list

    def get_one_day_values(self):
        """
        获取一天的ndarray
        :return:
        """
        filter_UITs = self.all_UITs[self.all_UITs[:, 3] == self.cur_day]

        if self.if_disp:
            print("cur day {}".format(self.cur_day), end=" , ")
            print("total events {}".format(filter_UITs.shape[0]))
        return filter_UITs

    def extract_feature(self, item_id):
        """
        通过item_id获取视频的特征
        :param item_id:
        :return:
        """
        if self.cur_day > self.max_day:
            raise Exception("days exceed")

        cur_item = self.item_list[item_id]
        if np.sum(cur_item.history) == 0:
            return np.zeros((self.dim,), dtype=np.int32)

        feature = [0] * self.dim
        for pos, feature_day in enumerate(feature_day_list):
            if self.cur_day >= feature_day:
                check = np.sum(cur_item.history[self.cur_day - feature_day:self.cur_day])
            else:
                check = np.sum(cur_item.history[0:self.cur_day])
            if check > 0:
                feature[pos] = 1

        if self.if_disp:
            print("get the feature: ", np.array(feature, dtype=np.int32))

        return np.array(feature, dtype=np.int32)

    def update_feature(self):
        """
        根据ndarray更新当天视频的特征值
        :return:
        """
        count = 0
        for value_line in self.curday_values:
            item_id = value_line[2]
            self.item_list[item_id].history[self.cur_day] += 1
            count += 1

        item_count = 0
        for item in self.item_list:
            if item.history[self.cur_day]:
                item.popularity = item.history[self.cur_day]
                item_count += 1

        if self.if_disp:
            print("total deal with {} events, update {} items".format(count, item_count))

        return item_count

    def update_hypercube_estimate_value(self):
        """
        根据event_list列表更新hypercube他们的MN值
        :return:
        """

        total_popularity = 0

        if self.if_disp:
            print("update hypercube M N")

        for event in self.event_lastday_list:
            video_id = event.item
            event_feature = self.extract_feature(video_id)
            cube = self.select_hypercube(event_feature)
            popularity_plus = cube.update_popularity(self.item_list[video_id].popularity)
            total_popularity += popularity_plus

            if self.if_disp:
                print("current hypercube ", event_feature, end=" , ")
                print("popularity plus {}".format(popularity_plus))

        return total_popularity

    def update_cache_set(self):
        tmp_list = PriorityQueue()
        while not self.cache_list.empty():
            (old_priority, old_item) = self.cache_list.get()
            event_feature = self.extract_feature(old_item)
            cube = self.select_hypercube(event_feature)
            update_estimate_popularity = cube.get_popularity()
            tmp_list.put((update_estimate_popularity, old_item))

        if tmp_list.qsize() != self.cache_set.__len__():
            raise Exception("update heap error")

        self.cache_list = tmp_list

        for event in self.event_lastday_list:
            if event.item in self.cache_set:
                continue

            if len(self.cache_set) < self.cache_size:
                self.cache_set.add(event.item)
                self.cache_list.put((event.esti_popularity, event.item))
            else:
                (top_priority, top_item) = self.cache_list.queue[0]

                if event.esti_popularity >= top_priority:  # 替换条件
                    (replace_popularity, replace_item) = self.cache_list.get()
                    self.cache_set.remove(replace_item)
                    self.cache_list.put((event.esti_popularity, event.item))
                    self.cache_set.add(event.item)
                    # print("{} out and {} in".format(replace_popularity, event.esti_popularity))
        return True

    def update_one_day(self):
        """
        天数加1
        :return:
        """
        self.cur_day += 1
        self.event_lastday_list = []
        if self.cur_day > self.max_day:
            raise Exception("days exceed")

        if self.if_disp:
            print("one day adances, current day {}".format(self.cur_day))

        return self.cur_day

    def select_hypercube(self, feature):
        """
        根据视频的特征选择对应的hypercube
        :param feature:
        :return:
        """
        hypercube = self.hypercube_array
        for i in feature:
            hypercube = hypercube[i]

        assert isinstance(hypercube, Hypercube)

        if self.if_disp:
            print("current hypercube: ", feature)

        return hypercube

    def estimate_popularity(self, hypercube, event):
        """
        根据hypercube的总体估计当前事件的流行度
        :param hypercube:
        :param event_id:
        :return:
        """
        hypercube.add_event(event)       # 添加一个事件
        popularity = hypercube.get_popularity()     # 获取popularity
        event.esti_popularity = popularity

        if self.if_disp:
            print("the esti_popularity is {}".format(popularity))

        return popularity

    def event_add_oneday(self,curValues):
        """
        添加cur_day当天的event进event_list
        """
        #self.curday_values = self.get_one_day_values()
        self.curday_values = curValues
        for value_line in self.curday_values:
            self.event_count += 1
            event = Event(id=value_line[0], user=value_line[1], item=value_line[2], occur_time=value_line[3], esti_popularity=0)
            self.event_lastday_list.append(event)

        if self.if_disp:
            print("today add {} events, current event pos: {}".format(self.curday_values.shape[0], self.event_count))    # for check use

        return self.event_lastday_list

    def curday_event_into_cube(self, is_validate=False):
        count = 0
        hit = 0
        for event in self.event_lastday_list:

            if self.if_disp:
                print("estimate event {}".format(event.id))

            event_feature = self.extract_feature(event.item)
            cube = self.select_hypercube(event_feature)
            _estimate_popularity = self.estimate_popularity(cube, event)

            hit += self.metric(event)
            count += 1

        if self.if_disp:
            print("total estimate {} events".format(count))

        if is_validate:
            print("today hit rate is {}".format(round(hit/count, 3)))

        return count, hit

    def print_cubes(self):
        print("day {} show cubes".format(self.cur_day))
        hypercubes_list = self.enumerate_hypercubes()

        for pos, hypercube in enumerate(hypercubes_list):
            print("current cube {}".format(pos), end=" , ")
            print("M:{} N:{}".format(hypercube.M, hypercube.N), end=" , ")
            print(hypercube.get_popularity())

    def metric(self, event):
        """
        最小堆比较，缓存击中返回1，否则返回0；
        :param event:
        :return:
        """
        hit = 0
        if event.item not in self.cache_set:
            # if len(self.cache_set) < self.cache_size:
            #     self.cache_set.add(event.item)
            #     self.cache_list.put((event.esti_popularity, event.item))
            # else:
            #     (top_priority, top_item) = self.cache_list.queue[0]
            #
            #     if event.esti_popularity >= top_priority:  # 替换条件
            #         (replace_popularity, replace_item) = self.cache_list.get()
            #         self.cache_set.remove(replace_item)
            #         self.cache_list.put((event.esti_popularity, event.item))
            #         self.cache_set.add(event.item)
                    # print("{} out and {} in".format(replace_popularity, event.esti_popularity))
            return hit
        else:
            hit = 1
            return 1

In [3]:
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

In [4]:
class ENV(object):
    def __init__(self,userNum,contentNum):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum)
        self.e = np.zeros(shape=contentNum)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l_edge = 0.1
        self.l_cp = 1

        self.B = np.full(shape=userNum,fill_value=15,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):

        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)

        self.r[u] = self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (torch.from_numpy(self.r),
                torch.from_numpy(self.p) , 
                torch.from_numpy(self.e),
                torch.from_numpy(self.S),
                self.l_edge,
                self.l_cp)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()


In [5]:
level = 'u'
# flag = "不按天更新"
#data_path = '/home/zhangxz/workspace/data/Nati1000_U500_V30/'
#data_path = '/home/ubuntu/data/dataset/R1584_U50_V2/'
data_path = '/home/ubuntu/data/dataset/R3009_U5_V100/'
#part_one = pd.read_csv(data_path + 'train.csv', header=None)
#part_two = pd.read_csv(data_path + 'test.csv', header=None)
#UIT = pd.concat([part_one, part_two], axis=0)
UIT = pd.read_csv(data_path + 'UIT.csv')

#import os
data_path = '/home/ubuntu/data/dataset/R3009_U5_V100_header/'
if not os.path.exists(data_path):
    os.makedirs(data_path)

UIT.to_csv(data_path + 'UIT.csv',index=0,header=None)
UIT = pd.read_csv(data_path + 'UIT.csv',header=None)

#UIT = UIT[UIT['day']<18]
group_count = 0
result_list = []
#UIT.columns = [[0,1,2,3,4,5,6,7,8,9,10,11]]
UIT

,0,1,2,3,4,5,6,7,8,9,10,11
0,365,3391,0,0,148,11,21,0,0,8812,1,0
1,203,5779,0,0,7,11,4,0,0,9063,1,0
2,208,4675,0,0,92,13,4,0,0,3444,1,0
3,159,332,0,0,56,11,3,0,0,4058,1,0
4,50,674,0,0,439,11,4,0,0,1549,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
300978,483,6831,29,2591880,34,11,4,0,0,7061,1,0
300979,158,8448,29,2591880,34,11,4,0,0,11316,1,0
300980,483,6463,29,2591940,35,11,4,0,0,7061,1,0
300981,158,4715,29,2591940,34,11,4,0,0,11316,1,0


In [6]:
trainUIT = UIT[UIT[2]<18]
contentNum = max(UIT[1])+1
userNum = max(UIT[0])+1
UIT[2] = UIT[3]//(60*60)
contentNum,userNum,UIT

(10000,
 500,
          0     1    2        3    4     5       6   7   8      9   10  11
 0       365  3391    0        0  148  1030  101001   0   0  11807   1   2
 1       203  5779    0        0    7  1030   10203   0   0  15068   1   2
 2       208  4675    0        0   92  1035   10203   0   0   5375   1   2
 3       159   332    0        0   56  1030   10202   0   0   5992   1   2
 4        50   674    0        0  439  1030   10203   0   0   3468   1   2
 ...     ...   ...  ...      ...  ...   ...     ...  ..  ..    ...  ..  ..
 300978  483  6831  719  2591880   34  1030   10203   0   0  10010   1   2
 300979  158  8448  719  2591880   34  1030   10203   0   0  23340   1   2
 300980  483  6463  719  2591940   35  1030   10203   0   0  10010   1   2
 300981  158  4715  719  2591940   34  1030   10203   0   0  23340   1   2
 300982  483  2021  719  2591940   34  1030   10203   0   0  10010   1   2
 
 [300983 rows x 12 columns])

In [7]:
UIT_index = UIT.reset_index()
UIT_index

,index,0,1,2,3,4,5,6,7,8,9,10,11
0,0,365,3391,0,0,148,1030,101001,0,0,11807,1,2
1,1,203,5779,0,0,7,1030,10203,0,0,15068,1,2
2,2,208,4675,0,0,92,1035,10203,0,0,5375,1,2
3,3,159,332,0,0,56,1030,10202,0,0,5992,1,2
4,4,50,674,0,0,439,1030,10203,0,0,3468,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300978,300978,483,6831,719,2591880,34,1030,10203,0,0,10010,1,2
300979,300979,158,8448,719,2591880,34,1030,10203,0,0,23340,1,2
300980,300980,483,6463,719,2591940,35,1030,10203,0,0,10010,1,2
300981,300981,158,4715,719,2591940,34,1030,10203,0,0,23340,1,2


In [8]:
all_UITs = UIT.values
items_seq = max(all_UITs[:, 1] + 1)
index = np.arange(0, all_UITs.shape[0]).reshape((all_UITs.shape[0], 1))
all_UITs = np.column_stack((index, all_UITs))
all_UITs

array([[     0,    365,   3391, ...,  11807,      1,      2],
       [     1,    203,   5779, ...,  15068,      1,      2],
       [     2,    208,   4675, ...,   5375,      1,      2],
       ...,
       [300980,    483,   6463, ...,  10010,      1,      2],
       [300981,    158,   4715, ...,  23340,      1,      2],
       [300982,    483,   2021, ...,  10010,      1,      2]])

In [9]:
trainT = 18*24
testT = 12*24
days = trainT + testT
feature_day_list = [1,6,24,6*24]
print(UIT.shape[0],feature_day_list,days)
dim = 4


300983 [1, 6, 24, 144] 720


In [10]:
class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = torch.zeros(size=(env.contentNum,),dtype=int)

        self.Bu = int(env.B[self.u])
        self.contentNum = env.contentNum
        self.userNum = env.userNum

        self.r , self.p , self.e, self.S,self.l_edge, self.l_cp = env.getStatus()

        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.lastAction = self.action

        self.reward = 0
        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']

    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    
    def getReward(self,lastru,lastp,ru,p,i,action,S,Bu,l_edge,l_cp,e):

        self.Rh = - self.ALPHAh * (torch.log(lastru * lastp + (1-lastru) * (1-lastp)).sum() - torch.log(ru * p + (1-ru) * (1-p)).sum())

        self.Ro =   self.BETAo * action[i] * (S[i] / Bu + ( e[i] * l_edge + ( 1-e[i] ) * l_cp ) / S[i])

        self.Rl =   self.BETAl * ( ( 1 - action[i] )  * ( l_cp - ( e[i] * l_edge + ( 1 - e[i] ) * l_cp ) ) ) / S[i]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        return  self.Rh+self.Ro+self.Rl

    def selectAction(self,env,uit,popcache):

        self.lastAction = self.action
        self.lastp = self.p
        self.lastr = self.r

        self.updateViewContent(uit[1])
        self.r , self.p , self.e, self.S, self.l_edge, self.l_cp = env.getStatus()
        
        self.reward = self.getReward(self.lastr[self.u],self.lastp,self.r[self.u],self.p,self.W[-1],self.lastAction,self.S,self.Bu,self.l_edge,self.l_cp,self.e)
        
        cacheSet = popcache.cache_set.copy()
        
        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.action[self.W[-1]] = 1
        if len(cacheSet)<self.Bu:
            actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
            if self.W[-1] not in actionIndex:
                actionIndex.pop()
            for index in actionIndex:
                self.action[index] = 1
        else:
            if self.W[-1] not in cacheSet:
                cacheSet.pop() 
            for i in cacheSet:
                self.action[i] = 1

        env.updateEnv(self.u,self.action.numpy(),uit[2])

        #print(self.W[-1],np.argwhere(ue.action.numpy()==1),cacheSet)
        return self.action

In [11]:
UEs = {}
rewardPara = {"alpha":1,"betao":0.5,"betal":0.5}
env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0
curDay = 0

cache_size_ratio =0.0015
cache_size = int(contentNum * cache_size_ratio)
popcache = Popcache(dim, max_item_id = contentNum, max_day=days, all_UITs=all_UITs,cur_day=curDay,cache_size=cache_size,if_disp=False)

eventIndex = 0
oneDayEvents = []

In [12]:
sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)

for index,trace in UIT.iterrows():
    
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    actionIndex = np.argwhere(ue.lastAction.numpy())
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1

    ue.selectAction(env=env,uit=uit,popcache=popcache)
    
    actionIndex = np.argwhere(ue.action.numpy())
    for content in actionIndex:
        oneDayEvents.append([eventIndex,uit[0],content,uit[2]])
        eventIndex += 1 
    
    if curDay<uit[2]:
        #print(curDay)
        #print(len(UIT_index[UIT_index[2]==curDay-1].values))
        event_list = popcache.event_add_oneday(np.array(oneDayEvents,dtype=int))
        curDay = uit[2]
        click, hit = popcache.curday_event_into_cube()
        popcache.update_feature()
        total_popularity = popcache.update_hypercube_estimate_value()
        popcache.update_cache_set()
        # popcache.print_cubes()
        # popcache.enumerate_hypercubes()
        popcache.update_one_day()
        popcache.cur_day = curDay
        oneDayEvents = []
        


    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if index % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5),"privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Wed Sep 22 17:25:31 2021   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 4.02118

--Time: Wed Sep 22 17:25:56 2021   Index: 10000   Loss: 0.0 --
Reward: [0.99005 0.16617 0.00122] total reward: 1.15744
UEHitrate: 0.0036  edgeHitrate 0.37016 sumHitrate 0.37376  privacy: 1.36287

--Time: Wed Sep 22 17:26:21 2021   Index: 20000   Loss: 0.0 --
Reward: [0.67058 0.18577 0.00146] total reward: 0.85782
UEHitrate: 0.003  edgeHitrate 0.41358 sumHitrate 0.41658  privacy: 1.1479

--Time: Wed Sep 22 17:26:46 2021   Index: 30000   Loss: 0.0 --
Reward: [0.52698 0.1793  0.00169] total reward: 0.70797
UEHitrate: 0.00383  edgeHitrate 0.39935 sumHitrate 0.40319  privacy: 1.07913

--Time: Wed Sep 22 17:27:16 2021   Index: 40000   Loss: 0.0 --
Reward: [0.43625 0.1739  0.00176] total reward: 0.61191
UEHitrate: 0.00415  edgeHitrate 0.38729 sumHitrate 0.39144  privacy: 1.03739

--Time: Wed Sep 22 17:27:47 2021   Index: 50000   Loss

In [13]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.     , 0.37376, 0.41658, 0.40319, 0.39144, 0.38871, 0.38539,
        0.37814, 0.38167, 0.37976, 0.3794 , 0.38014, 0.38003, 0.37633,
        0.38026, 0.38454, 0.38198, 0.38524, 0.38771, 0.3892 , 0.38966,
        0.39153, 0.38982, 0.38852, 0.38749, 0.38799, 0.3862 , 0.38439,
        0.38446, 0.38683, 0.39059, 0.     ]),
 array([0.     , 0.0036 , 0.003  , 0.00383, 0.00415, 0.00456, 0.00442,
        0.00451, 0.00452, 0.00468, 0.00488, 0.0053 , 0.00562, 0.00615,
        0.00633, 0.00653, 0.00662, 0.00672, 0.00676, 0.007  , 0.00699,
        0.00721, 0.00729, 0.00747, 0.00766, 0.00776, 0.00784, 0.00795,
        0.00797, 0.00798, 0.00813, 0.     ]),
 array([0.     , 0.37016, 0.41358, 0.39935, 0.38729, 0.38415, 0.38098,
        0.37362, 0.37715, 0.37508, 0.37452, 0.37484, 0.37441, 0.37017,
        0.37393, 0.378  , 0.37536, 0.37853, 0.38095, 0.3822 , 0.38267,
        0.38431, 0.38253, 0.38105, 0.37982, 0.38023, 0.37836, 0.37644,
        0.37648, 0.37885, 0.38246, 0.     ]))

In [14]:
privacyReduction

array([4.02118, 1.36287, 1.1479 , 1.07913, 1.03739, 1.01468, 1.00241,
       0.9926 , 0.98298, 0.9759 , 0.96339, 0.95134, 0.94805, 0.94324,
       0.93802, 0.93874, 0.93687, 0.93642, 0.93009, 0.92723, 0.92409,
       0.91341, 0.91185, 0.91044, 0.90883, 0.90863, 0.907  , 0.90567,
       0.90533, 0.90564, 0.90619, 0.     ])

In [15]:
UEs = {}
rewardPara = {"alpha":1,"betao":0.5,"betal":0.5}
env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0
curDay = 0

cache_size_ratio =0.0015
cache_size = int(contentNum * cache_size_ratio)
popcache = Popcache(dim, max_item_id = contentNum, max_day=days, all_UITs=all_UITs,cur_day=curDay,cache_size=cache_size,if_disp=False)

eventIndex = 0
oneDayEvents = []

In [16]:
sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)

for index,trace in UIT.iterrows():
    
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    actionIndex = np.argwhere(ue.lastAction.numpy())
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1

    ue.selectAction(env=env,uit=uit,popcache=popcache)
    
    actionIndex = np.argwhere(ue.action.numpy())
    for content in actionIndex:
        oneDayEvents.append([eventIndex,uit[0],content,uit[2]])
        eventIndex += 1 

    #print(curDay)
    #print(len(UIT_index[UIT_index[2]==curDay-1].values))
    event_list = popcache.event_add_oneday(np.array(oneDayEvents,dtype=int))
    curDay = uit[2]
    click, hit = popcache.curday_event_into_cube()
    popcache.update_feature()
    total_popularity = popcache.update_hypercube_estimate_value()
    popcache.update_cache_set()
    # popcache.print_cubes()
    # popcache.enumerate_hypercubes()
    #popcache.update_one_day()
    popcache.event_lastday_list = []
    popcache.cur_day = curDay
    oneDayEvents = []
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if index % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5),"privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Wed Sep 22 17:45:29 2021   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 4.02118

--Time: Wed Sep 22 17:46:36 2021   Index: 10000   Loss: 0.0 --
Reward: [2.10529e+00 1.67560e-01 1.24000e-03] total reward: 2.27409
UEHitrate: 0.0037  edgeHitrate 0.37356 sumHitrate 0.37726  privacy: 1.89026

--Time: Wed Sep 22 17:47:43 2021   Index: 20000   Loss: 0.0 --
Reward: [1.58613 0.18593 0.00168] total reward: 1.77375
UEHitrate: 0.00415  edgeHitrate 0.41443 sumHitrate 0.41858  privacy: 1.86039

--Time: Wed Sep 22 17:48:49 2021   Index: 30000   Loss: 0.0 --
Reward: [1.33087 0.17906 0.00193] total reward: 1.51187
UEHitrate: 0.005  edgeHitrate 0.39949 sumHitrate 0.40449  privacy: 1.89512

--Time: Wed Sep 22 17:49:56 2021   Index: 40000   Loss: 0.0 --
Reward: [1.14319 0.1735  0.00197] total reward: 1.31866
UEHitrate: 0.00555  edgeHitrate 0.38692 sumHitrate 0.39247  privacy: 1.87911

--Time: Wed Sep 22 17:51:03 2021   Index:

In [6]:
hitrate1 = np.array([0.0107, 0.02, 0.0349, 0.0539 ,0.1065, 0.2135, 0.3443, 0.482,0.6706, 0.7719, 0.8585, 0.9425])
hitrate2 =np.array([0.0112, 0.0206, 0.0354, 0.0545, 0.1038, 0.2096, 0.3406, 0.4616,0.6197, 0.7354, 0.8397, 0.9395])
hitrate3 = np.array([0.0111, 0.0218, 0.0378, 0.056,0.1117, 0.2306, 0.3692, 0.4896,0.6159, 0.7307, 0.8357, 0.9388])
(UIT[(UIT[2]>=12)&(UIT[2]<18)].shape[0]*hitrate1+UIT[(UIT[2]>=18)&(UIT[2]<24)].shape[0]*hitrate2+UIT[(UIT[2]>=24)&(UIT[2]<30)].shape[0]*hitrate3)/UIT[(UIT[2]>=12)&(UIT[2]<30)].shape[0]

array([0.01091909, 0.02026291, 0.03511909, 0.05416291, 0.1053169 ,
       0.21179107, 0.34267871, 0.47306099, 0.64829629, 0.75590618,
       0.85026209, 0.94118544])

In [8]:
np.set_printoptions(linewidth=170)
hitrate1 = np.array([0.02222, 0.04083, 0.05233, 0.08623, 0.15234, 0.25517, 0.34928, 0.43627, 0.5698, 0.69672, 0.78608, 0.84388])
hitrate2 =np.array([0.02245, 0.03751, 0.04835, 0.06754, 0.10708, 0.19111, 0.25089, 0.31116, 0.43444, 0.57156, 0.64897, 0.72132])
hitrate3 = np.array([0.01563, 0.0258, 0.0496, 0.05668, 0.13091, 0.25474, 0.33844, 0.40537, 0.50362, 0.60923, 0.6901, 0.76312])
(UIT[(UIT[2]>=12)&(UIT[2]<18)].shape[0]*hitrate1+UIT[(UIT[2]>=18)&(UIT[2]<24)].shape[0]*hitrate2+UIT[(UIT[2]>=24)&(UIT[2]<30)].shape[0]*hitrate3)/UIT[(UIT[2]>=12)&(UIT[2]<30)].shape[0]

array([0.02232078, 0.03937522, 0.05058602, 0.07804029, 0.13250767, 0.22709976, 0.30616681, 0.38144846, 0.51048704, 0.64187655, 0.72600022, 0.79017583])